In [10]:
import os 
from glob import glob
from PIL import Image
import numpy as np

#https://medium.com/trackin-datalabs/data-input-%EB%A7%8C%EB%93%A4%EA%B8%B0-74bb5c1ce52f

#전처리 기술
data_list = glob('data\\trainingSet\\*\\*.jpg') 


def get_label_from_path(path):
    return int(path.split('\\')[-2])

def read_image(path):
    image = np.array(Image.open(path))
    print(path)
    print(image.shape[0])
    print(type(image))
    return image.reshape(image.shape[0], image.shape[1])

label_name_list= []

for path in data_list:
    label_name_list.append(get_label_from_path(path))
    

#원핫인코딩

unique_label_names = np.unique(label_name_list)
def onehot_encode_label(path):
    onehot_label = unique_label_names == get_label_from_path(path)
    onehot_label = onehot_label.astype(np.uint8)
    return onehot_label

link = data_list[11311]
print(get_label_from_path(link))
read_im = read_image(link)
pi = Image.fromarray(np.int32(read_im))
print(pi)
## pi.show()


#배치만들기
import tensorflow as tf

def _read_py_function(path, label):
    image = read_image(path)
    label = np.array(label, dtype=np.uint8)
    return image.astype(np.int32), label

def _resize_function(image_decoded, label):
    image_decoded.set_shape([None, None, None])
    image_resized = tf.image.resize_images(image_decoded, [28, 28])
    print(image_resized)
    return image_resized, label

# label을 array 통채로 넣는게 아니고, list 화 시켜서 하나씩 넣기 위해 list로 바꿔주었다. 

batch_size =500
label_list = [onehot_encode_label(path).tolist() for path in data_list]


dataset = tf.data.Dataset.from_tensor_slices((data_list, label_list))
dataset = dataset.map(lambda data_list, label_list: tuple(tf.py_func(_read_py_function, [data_list, label_list], [tf.int32, tf.uint8])))
#tf.py_func -> 인수1: 처리할 함수 인수2: 입력값 인수3: 반환형식
dataset = dataset.map(_resize_function)
dataset = dataset.repeat(100) #epoch 의 수를 정한다
dataset = dataset.shuffle(buffer_size=(int(len(data_list) * 0.4) + 3 * batch_size)) # 섞을 정도

dataset = dataset.batch(batch_size)# 아주 편리한 배치만들기 방식

iterator = dataset.make_initializable_iterator()
image_stacked, label_stacked = iterator.get_next()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)

2
data\trainingSet\2\img_32697.jpg
28
<class 'numpy.ndarray'>
<PIL.Image.Image image mode=I size=28x28 at 0x1DDE44C94A8>
Tensor("resize_images/Squeeze:0", shape=(28, 28, ?), dtype=float32)
